In [1]:
import requests

API_URL = "http://localhost:5433"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

def generate(prompt: str) -> str:
    """เรียก /api/generate เพื่อสร้างข้อความตอบกลับ"""
    res = requests.post(
        f"{API_URL}/api/generate",
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )
    res.raise_for_status()
    return res.json()["response"]

if __name__ == "__main__":
    print(generate("สวัสดี"))

สวัสดีครับ! ยินดีที่ได้รู้จักครับ ผมชื่อ Typhoon สร้างโดย SCB 10X เพื่อช่วยเหลือคุณอย่างมีประโยชน์ ปลอดภัย และซื่อสัตย์

มีอะไรให้ผมช่วยวันนี้ครับ?


In [21]:
import os

def load_and_chunk(directory: str, chunk_size: int = 1600):
    """อ่านไฟล์ .txt ในโฟลเดอร์ แล้วแบ่งเป็นชิ้นละ chunk_size คำ"""
    chunks = []
    for fname in os.listdir(directory):
        if not fname.endswith(".txt"):
            continue
        text = open(os.path.join(directory, fname), encoding="utf-8").read()
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            # เก็บ metadata เบื้องต้นด้วย
            chunks.append({
                "content": chunk,
                "source": fname
            })
    return chunks

# usage
directory = "Raw-data-from-TTT"
list_of_chunks = load_and_chunk(directory)
print(f"ได้ทั้งหมด {len(list_of_chunks)} chunks")


ได้ทั้งหมด 15 chunks


In [16]:
from sentence_transformers import SentenceTransformer

# โหลดโมเดล embedding
embed_model = SentenceTransformer("BAAI/bge-m3")

In [ ]:
# แปลงเป็นเวกเตอร์
texts = [c["content"] for c in list_of_chunks]
embeddings = embed_model.encode(texts, show_progress_bar=True)

# เพิ่ม embedding ลงใน list_of_chunks
for chunk, emb in zip(list_of_chunks, embeddings):
    chunk["vector"] = emb.tolist()

Batches: 100%|██████████| 1/1 [00:11<00:00, 11.64s/it]


In [23]:
import psycopg2

conn = psycopg2.connect(
    dbname="mydb", user="admin", password="1234",
    host="localhost", port="5432"
)
cur = conn.cursor()
# สร้างตารางถ้ายังไม่มี
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        source TEXT,
        embedding VECTOR(1024)
    )
""")
conn.commit()

# แทรกข้อมูล
for c in list_of_chunks:
    cur.execute("""
        INSERT INTO documents (content, source, embedding)
        VALUES (%s, %s, %s)
    """, (c["content"], c["source"], c["vector"]))
conn.commit()
cur.close()
conn.close()


In [ ]:
import psycopg2
from sentence_transformers import SentenceTransformer
import requests

# โหลดโมเดล embedding
embed_model = SentenceTransformer("BAAI/bge-m3")

# 1. ฟังก์ชันสืบค้นเอกสาร
def retrieve_similar(query: str, top_k: int = 7):
    """
    รับข้อความค้นหา (query) -> แปลงเป็นเวกเตอร์ -> 
    SELECT เอกสารที่ embedding ใกล้เคียงที่สุด top_k ชิ้น
    """
    # แปลง query เป็นเวกเตอร์
    q_vec = embed_model.encode([query], show_progress_bar=False)[0].tolist()

    # เชื่อมต่อฐานข้อมูล
    conn = psycopg2.connect(
        dbname="mydb", user="admin", password="1234",
        host="localhost", port="5432"
    )
    cur = conn.cursor()

    # สืบค้น top_k เอกสาร
    cur.execute("""
        SELECT content, source
        FROM documents
        ORDER BY embedding <-> (%s::vector)
        LIMIT %s
    """, (q_vec, top_k))

    rows = cur.fetchall()
    cur.close()
    conn.close()

    return [{"content": r[0], "source": r[1]} for r in rows]


API_URL = "http://localhost:5433"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

# 2. ฟังก์ชันเรียก LLM
def generate_rag_response(query: str, top_k: int = 7) -> str:
    # 2.1 สืบค้นเอกสาร
    docs = retrieve_similar(query, top_k)

    # 2.2 ประกอบ context
    context = "\n\n".join(
        f"Source: {doc['source']}\n{doc['content']}" for doc in docs
    )
    prompt = (
        "คุณคือผู้ช่วยอัจฉริยะชื่อ TTT-Assistant จากบริษัท ทีทีที บราเธอร์ส จำกัด "
        "กรุณาตอบคำถามให้ละเอียดและครบถ้วนที่สุด โดยใช้ข้อมูลจาก context ที่มี\n"
        f"---\nContext: {context}\n---\n"
        f"Question: {query}\n"
        "Answer:"
    )

    # 2.3 เรียก LLM API
    res = requests.post(
        f"{API_URL}/api/generate",
        json={"model": MODEL_NAME, "prompt": prompt, "stream": False},
        timeout=240
    )
    res.raise_for_status()
    return res.json()["response"]


# 3. วิธีใช้งาน
if __name__ == "__main__":
    question = "วันหยุดที่ตรงกับวันจันทร์ทั้งหมด"
    answer = generate_rag_response(question)
    print(answer)

จากข้อมูลที่ให้มา มีวันหยุดที่ตรงกับวันจันทร์ดังนี้:

*   วันจันทร์ที่ 7 เมษายน – วันหยุดชดเชยวันจักรี (อาทิตย์ที่ 6 เมษายน 2568)
*   วันจันทร์ที่ 12 พฤษภาคม – วันหยุดชดเชยวันวิสาขบูชา (อาทิตย์ที่ 11 พฤษภาคม 2568)
*   วันจันทร์ที่ 13 ตุลาคม – วันนวมินทรมหาราช
